# covid stuff

----------------------------------------

- **created** by z: `2020-03-30`
- last **updated**: `2020-04-01T12:51:42PDT`

## _preamble_

#### import packages

In [216]:
import pathlib
import requests
import re
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

#### disable request warning

In [208]:
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#### directories

In [209]:
# cwd = pathlib.Path.cwd()
# cwd
# docs_dir = pathlib.Path("/private/var/mobile/Library/Mobile Documents/iCloud~AsheKube~Carnets/Documents")
# data_dir = docs_dir / "data"

# where data will be saved/loaded
data_dir = pathlib.Path("/Users/zarek/Dropbox/code/github/zcovid/data")
if not data_dir.is_dir():
    data_dir.mkdir()
    print(">>> created dir {}".format(data_dir))

#### URLs

In [210]:
# base URL for data downloads
base_tsdata_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data"

#### regexes

In [211]:
date_re = re.compile(r"\d+/\d+/\d+")

#### time-series keys

In [212]:
# for file names and saving data
tskeys = [
    'confirmed_US',
    'confirmed_global',
    'deaths_US',
    'deaths_global',
    'recovered_global'
]

#### utility functions

In [253]:
# format number
SINUM_PREFIXES = {
    -6: {}'a',    # atto
    -5: 'f',    # femto
    -4: 'p',    # pico
    -3: 'n',    # nano
    -2: 'μ',    # micro
    -1: 'm',    # milli
    0: ' ',     # -
    1: 'k',     # kilo
    2: 'M',     # mega
    3: 'G',     # giga
    4: 'T',     # tera
    5: 'P',     # peta
    6: 'E'      # exa
}
def sinum(num, unit='B', fmt="{coef:.3f} {pfx}{unit}", long_pfx=False, binary=False, verbose=False):
    assert isinstance(num, (int, float))
    assert isinstance(unit, str)
    # verbosity...
    if verbose:
        print(f">>> sinum(num={num!r}, unit={unit!r}, fmt={fmt!r}, binary={binary}, verbose={verbose})")
    def _verb(name, value):
        if verbose:
            print("\t{:<12s} = {:>20s}".format(
                name, 
                value if isinstance(value, str) else f"{value:g}"
            ))
    # binary mods
    if binary:
        log_base = 1024
        unit = f"i{unit!s}"        
    else:
        log_base = 1000
        unit = str(unit)
    # order of magnitude
    oom = math.floor(math.log(num, log_base))
    _verb('oom', oom)
    # coefficient
    coef = num / (log_base ** oom)
    _verb('coef', coef)
    # SI prefix
    pfx = SINUM_PREFIXES[oom]
    _verb('pfx', pfx)
    # string out
    out = fmt.format(coef=coef, pfx=pfx, unit=unit)
    _verb('out', out)

    return out


# tests
print(sinum(82457891234, verbose=True), end='\n\n')
print(sinum(82457891234, verbose=True, unit=''), end='\n\n')
print(sinum(82457891234, verbose=True, unit='bloops'), end='\n\n')
print(sinum(82457891234, binary=True, verbose=True), end='\n\n')
print(sinum(0.00082457891234, verbose=True), end='\n\n')
print(sinum(824578912342345.2345, verbose=True), end='\n\n')
print(sinum(0.00082457891234, binary=True, verbose=True), end='\n\n')
print(sinum(824578912342345.2345, binary=True, verbose=True), end='\n\n')
print(sinum(824578912342345.2345, binary=True, verbose=True, fmt="{coef:12.5f} // {pfx} // {unit}"), end='\n\n')







>>> sinum(num=82457891234, unit='B', fmt='{coef:.3f} {pfx}{unit}', binary=False, verbose=True)
	oom          =                    3
	coef         =              82.4579
	pfx          =                    G
	out          =            82.458 GB
82.458 GB

>>> sinum(num=82457891234, unit='', fmt='{coef:.3f} {pfx}{unit}', binary=False, verbose=True)
	oom          =                    3
	coef         =              82.4579
	pfx          =                    G
	out          =             82.458 G
82.458 G

>>> sinum(num=82457891234, unit='bloops', fmt='{coef:.3f} {pfx}{unit}', binary=False, verbose=True)
	oom          =                    3
	coef         =              82.4579
	pfx          =                    G
	out          =       82.458 Gbloops
82.458 Gbloops

>>> sinum(num=82457891234, unit='B', fmt='{coef:.3f} {pfx}{unit}', binary=True, verbose=True)
	oom          =                    3
	coef         =              76.7949
	pfx          =                    G
	out          =          

## load data

#### initialize data container `d`

In [213]:
# a dict with tskeys as keys to uniform dicts
for tsk in tskeys:
    d[tsk] = {}
d

{'confirmed_US': {},
 'confirmed_global': {},
 'deaths_US': {},
 'deaths_global': {},
 'recovered_global': {}}

#### download data from github, save to file

In [215]:
# same load and save process for each tskey
print(f">>> loading CSVs, saving in ``{data_dir}''\n")
for tsk in tskeys:
    print(">>> getting data for '{}'".format(tsk))
    d[tsk]['url'] = f"{base_tsdata_url}/csse_covid_19_time_series/time_series_covid19_{tsk}.csv"
    d[tsk]['req'] = requests.get(d[tsk]['url'], auth=('user', 'pass'))
    d[tsk]['raw'] = d[tsk]['req'].content
    d[tsk]['csv'] = data_dir / f"time_series_covid19_{tsk}.csv"
    if d[tsk]['csv'].is_file() and d[tsk]['csv'].stat().st_size > 0:
        print(f"--> CSV for '{tsk}' will be overwritten")
    with d[tsk]['csv'].open('w') as f:
        print("--> writing ``.../{}'' ... ".format(d[tsk]['csv'].name), end='')
        f.write(d[tsk]['raw'].decode())
        print("wrote {} B\n".format(d[tsk]['csv'].stat().st_size))
    del d[tsk]['raw']
# d

>>> loading CSVs, saving in ``/Users/zarek/Dropbox/code/github/zcovid/data''

>>> getting data for 'confirmed_US'
--> CSV for 'confirmed_US' will be overwritten
--> writing ``.../time_series_covid19_confirmed_US.csv'' ... wrote 766891 B

>>> getting data for 'confirmed_global'
--> CSV for 'confirmed_global' will be overwritten
--> writing ``.../time_series_covid19_confirmed_global.csv'' ... wrote 53769 B

>>> getting data for 'deaths_US'
--> CSV for 'deaths_US' will be overwritten
--> writing ``.../time_series_covid19_deaths_US.csv'' ... wrote 779130 B

>>> getting data for 'deaths_global'
--> CSV for 'deaths_global' will be overwritten
--> writing ``.../time_series_covid19_deaths_global.csv'' ... wrote 44961 B

>>> getting data for 'recovered_global'
--> CSV for 'recovered_global' will be overwritten
--> writing ``.../time_series_covid19_recovered_global.csv'' ... wrote 45985 B



#### load data from CSV just saved

In [190]:
# iterate tskeys, loading from CSV saved above
for tsk in tskeys:
    d[tsk]['df'] = pd.read_csv(d[tsk]['csv'])
    print(f"--> read CSV data for {tsk}")
# create backup of d
d_BAK = d.copy()

--> read CSV data for confirmed_US
--> read CSV data for confirmed_global
--> read CSV data for deaths_US
--> read CSV data for deaths_global
--> read CSV data for recovered_global


## clean up data

#### add index columns `d[tsk]['df']` (actual dataframe for the key), then reorder as desired

In [204]:
# column name substitutions
col_subs = {
    'Province_State': 'subregion',
    'Province/State': 'subregion',
    'Country_Region': 'region',
    'Country/Region': 'region',
    'Long_': 'long'    
} 

# columns to move to the beginning (in order)
priority_cols = [
    'locid',
    'region',
    'subregion',
    'combined_key'
]

# collect all columns since different dataframes dont have same columns
all_indx_cols = []
all_date_cols = []

# iterate through tskeys, cleaning up each
for tsk in tskeys:
    
    print(f">>> cleaning up dataframe for '{tsk}'")
    
    # add other index cols
    d[tsk]['df']['tskey'] = tsk
    d[tsk]['df']['domain'] = tsk.split('_')[1]
    d[tsk]['df']['datum'] = tsk.split('_')[0]
    d[tsk]['df']['locid'] = d[tsk]['df'].index
    
    d[tsk]['all_cols'] = list(d[tsk]['df'].columns)
    
    # clean up column names
    for i, c in enumerate(d[tsk]['all_cols']):
        if c in col_subs:
            c = col_subs[c]
        c = c.lower()
        d[tsk]['all_cols'][i] = c
    # print(d[tsk]['all_cols'])
    
    # get column subsets
    d[tsk]['df'].columns = d[tsk]['all_cols']
    d[tsk]['date_cols'] = list(filter(date_re.match, d[tsk]['all_cols']))
    d[tsk]['indx_cols'] = [i for i in d[tsk]['all_cols'] if i not in d[tsk]['date_cols']]

    # reorder columns
    col_idxs = list(range(len(d[tsk]['indx_cols'])))
    for col in priority_cols[::-1]:
        if col in d[tsk]['indx_cols']:
            idx = d[tsk]['indx_cols'].index(col)
            col_idxs.remove(idx)
            col_idxs.insert(0, idx)
    print(col_idxs)
    d[tsk]['indx_cols'] = [d[tsk]['indx_cols'][i] for i in col_idxs]
        
    # add to all_indx_cols
    for col in d[tsk]['indx_cols']:
        if col not in all_indx_cols:
            all_indx_cols.append(col)

    # add to all_date_cols
    for col in d[tsk]['date_cols']:
        if col not in all_date_cols:
            all_date_cols.append(col)

    # save dataframe with reordered columns
    d[tsk]['all_cols'] = [*d[tsk]['indx_cols'], *d[tsk]['date_cols']]
    d[tsk]['df'] = d[tsk]['df'][d[tsk]['all_cols']]

# d[tsk]

print(all_indx_cols)
print(all_date_cols)

d[tsk]['df']


>>> cleaning up dataframe for 'confirmed_US'


KeyError: 'df'

In [193]:
for tsk in tskeys:
    # print(d[tsk]['indx_cols'])
    # if type(d[tsk]['df'].columns).__name__ == 'Index':
    if ('df_BAK' not in d[tsk]) or isinstance(d[tsk]['df'].columns, pd.Index):
        d[tsk]['df_BAK'] = d[tsk]['df'].copy()

In [194]:
for tsk in tskeys:
    
    # create multiindex dataframe (df with just index cols)
    mindx_df = d[tsk]['df_BAK'][d[tsk]['indx_cols']]
    # create multiindex
    mindx = pd.MultiIndex.from_frame(mindx_df)
    # create new dataframe, old df transposed
    d[tsk]['df'] = d[tsk]['df_BAK'][d[tsk]['date_cols']].transpose()
    # add the new multiindex
    d[tsk]['df'].columns = mindx
    # convert index from str to datetime
    d[tsk]['df'].index = pd.to_datetime(d[tsk]['df'].index)
    
# mindx_df
# mindx

d[tskeys[0]]['df']

locid,0,1,2,3,4,5,6,7,8,9,...,3243,3244,3245,3246,3247,3248,3249,3250,3251,3252
region,US,US,US,US,US,US,US,US,US,US,...,US,US,US,US,US,US,US,US,US,US
subregion,American Samoa,Guam,Northern Mariana Islands,Puerto Rico,Virgin Islands,Alabama,Alabama,Alabama,Alabama,Alabama,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Grand Princess
combined_key,"American Samoa, US","Guam, US","Northern Mariana Islands, US","Puerto Rico, US","Virgin Islands, US","Autauga, Alabama, US","Baldwin, Alabama, US","Barbour, Alabama, US","Bibb, Alabama, US","Blount, Alabama, US",...,"Unassigned, Tennessee, US","Unassigned, Texas, US","Unassigned, Utah, US","Unassigned, Vermont, US","Unassigned, Virginia, US","Unassigned, Washington, US","Unassigned, West Virginia, US","Unassigned, Wisconsin, US","Unassigned, Wyoming, US","Grand Princess, US"
uid,16,316,580,630,850,84001001,84001003,84001005,84001007,84001009,...,84090047,84090048,84090049,84090050,84090051,84090053,84090054,84090055,84090056,84099999
iso2,AS,GU,MP,PR,VI,US,US,US,US,US,...,US,US,US,US,US,US,US,US,US,US
iso3,ASM,GUM,MNP,PRI,VIR,USA,USA,USA,USA,USA,...,USA,USA,USA,USA,USA,USA,USA,USA,USA,USA
code3,16,316,580,630,850,840,840,840,840,840,...,840,840,840,840,840,840,840,840,840,840
fips,60.0,66.0,69.0,72.0,78.0,1001.0,1003.0,1005.0,1007.0,1009.0,...,90047.0,90048.0,90049.0,90050.0,90051.0,90053.0,90054.0,90055.0,90056.0,99999.0
admin2,NaN,NaN,NaN,NaN,NaN,Autauga,Baldwin,Barbour,Bibb,Blount,...,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,NaN
lat,-14.271000,13.444300,15.097900,18.220800,18.335800,32.539527,30.727750,31.868263,32.996421,33.982109,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [78]:
tskeys

['confirmed_US',
 'confirmed_global',
 'deaths_US',
 'deaths_global',
 'recovered_global']

In [79]:
d['confirmed_US']['df'].join(d['confirmed_global']['df'])

/Users/zarek/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (15 levels on the left, 8 on the right)
  warnings.warn(msg, UserWarning)


,"(0, US, American Samoa, 16, AS, ASM, 16, 60.0, nan, -14.270999999999999, -170.132, American Samoa, US, confirmed_US, confirmed, US)","(1, US, Guam, 316, GU, GUM, 316, 66.0, nan, 13.4443, 144.7937, Guam, US, confirmed_US, confirmed, US)","(2, US, Northern Mariana Islands, 580, MP, MNP, 580, 69.0, nan, 15.0979, 145.6739, Northern Mariana Islands, US, confirmed_US, confirmed, US)","(3, US, Puerto Rico, 630, PR, PRI, 630, 72.0, nan, 18.2208, -66.5901, Puerto Rico, US, confirmed_US, confirmed, US)","(4, US, Virgin Islands, 850, VI, VIR, 850, 78.0, nan, 18.3358, -64.8963, Virgin Islands, US, confirmed_US, confirmed, US)","(5, US, Alabama, 84001001, US, USA, 840, 1001.0, Autauga, 32.53952745, -86.64408227, Autauga, Alabama, US, confirmed_US, confirmed, US)","(6, US, Alabama, 84001003, US, USA, 840, 1003.0, Baldwin, 30.72774991, -87.72207058, Baldwin, Alabama, US, confirmed_US, confirmed, US)","(7, US, Alabama, 84001005, US, USA, 840, 1005.0, Barbour, 31.868263, -85.3871286, Barbour, Alabama, US, confirmed_US, confirmed, US)","(8, US, Alabama, 84001007, US, USA, 840, 1007.0, Bibb, 32.99642064, -87.12511459999999, Bibb, Alabama, US, confirmed_US, confirmed, US)","(9, US, Alabama, 84001009, US, USA, 840, 1009.0, Blount, 33.98210918, -86.56790593, Blount, Alabama, US, confirmed_US, confirmed, US)",...,"(246, Canada, Yukon, 64.2823, -135.0, confirmed_global, confirmed, global)","(247, Kosovo, nan, 42.602636, 20.902977, confirmed_global, confirmed, global)","(248, Burma, nan, 21.9162, 95.956, confirmed_global, confirmed, global)","(249, United Kingdom, Anguilla, 18.2206, -63.0686, confirmed_global, confirmed, global)","(250, United Kingdom, British Virgin Islands, 18.4207, -64.64, confirmed_global, confirmed, global)","(251, United Kingdom, Turks and Caicos Islands, 21.69400000000001, -71.7979, confirmed_global, confirmed, global)","(252, MS Zaandam, nan, 0.0, 0.0, confirmed_global, confirmed, global)","(253, Botswana, nan, -22.3285, 24.6849, confirmed_global, confirmed, global)","(254, Burundi, nan, -3.3731, 29.9189, confirmed_global, confirmed, global)","(255, Sierra Leone, nan, 8.460555000000001, -11.779889, confirmed_global, confirmed, global)"
2020-01-22,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-26,0,45,0,64,17,6,5,0,0,2,...,3,71,0,0,0,0,0,0,0,0
2020-03-27,0,51,0,79,19,6,5,0,0,4,...,3,86,8,0,0,0,0,0,0,0
2020-03-28,0,55,0,100,22,6,10,0,0,5,...,4,91,8,2,2,4,2,0,0,0
2020-03-29,0,56,0,127,0,6,15,0,0,5,...,4,94,10,2,2,4,2,0,0,0


In [25]:
# ##### fignum = 0
# fig = plt.figure(fignum)
# plot_data = deaths_global.iloc[:, [0, 1, 2]]
# # plot_data.columns.names
# plot_regions = plot_data.columns[[0, 1, 2]].get_level_values('region')
# plt.plot(plot_data)
# plt.legend(plot_regions)